In [0]:
%pip install yellowbrick --quiet

In [0]:
import numpy as np
import pandas as pd
import datetime
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import colors
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from yellowbrick.cluster import KElbowVisualizer
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt, numpy as np
from mpl_toolkits.mplot3d import Axes3D
from sklearn.cluster import AgglomerativeClustering
from matplotlib.colors import ListedColormap
from sklearn import metrics
import warnings
import sys
if not sys.warnoptions:
    warnings.simplefilter("ignore")
np.random.seed(42)

In [0]:
sns.set(rc={"axes.facecolor":"#FFF9ED","figure.facecolor":"#FFF9ED"})
pallet = ["#682F2F", "#9E726F", "#D6B2B1", "#B9C0C9", "#9F8A78", "#F3AB60"]
cmap = colors.ListedColormap(["#682F2F", "#9E726F", "#D6B2B1", "#B9C0C9", "#9F8A78", "#F3AB60"])

In [0]:
df = pd.read_csv("Data/marketing_campaign.csv", sep="\t")
print(len(df))
display(df)

In [0]:
print(df['ID'].nunique())

# About the dataset

The dataset consists of **2,240 datapoints** and **29 attributes**.  
It can be categorized into the following subsets:

## Customer's Information
- **ID**
- **Year_Birth**
- **Education**
- **Marital_Status**
- **Income**
- **Kidhome**
- **Teenhome**
- **Dt_Customer**
- **Recency**
- **Complain**

## Products
*Amount spent on different products in the last 2 years:*
- **MntWines**
- **MntFruits**
- **MntMeatProducts**
- **MntFishProducts**
- **MntSweetProducts**
- **MntGoldProds**

## Promotion
- **NumDealsPurchases**
- **AcceptedCmp1**
- **AcceptedCmp2**
- **AcceptedCmp3**
- **AcceptedCmp4**
- **AcceptedCmp5**
- **Response**

## Place
- **NumWebPurchases**
- **NumCatalogPurchases**
- **NumStorePurchases**
- **NumWebVisitsMonth**


In [0]:
df.info()

In [0]:
df = df.dropna()
print(len(df))

In [0]:
df['Dt_Customer'] = pd.to_datetime(df['Dt_Customer'])

days = []
d1 = df['Dt_Customer'].max()
print(d1)

df["Customer_For"] = (d1 - df['Dt_Customer']).dt.days

display(df["Customer_For"])


In [0]:
print(df['Education'].value_counts())
print(df['Marital_Status'].value_counts())

In [0]:
display(df)

In [0]:
df['Age'] = 2025 - df['Year_Birth']

df['Spent'] = df["MntWines"]+ df["MntFruits"]+ df["MntMeatProducts"]+ df["MntFishProducts"]+ df["MntSweetProducts"]+ df["MntGoldProds"]

df['Living_With'] = df['Marital_Status'].replace({"Married":"Partner", "Together":"Partner", "Absurd":"Alone", "Widow":"Alone", "YOLO":"Alone", "Divorced":"Alone", "Single":"Alone",})

df['Children'] = df['Kidhome'] + df['Teenhome']

df["Family_Size"] = df['Living_With'].replace({"Alone": 1, "Partner": 2}) + df['Children']

df["Is_Parent"] = np.where(df.Children>0, 1, 0)

df["Education"]=df["Education"].replace({"Basic":"Undergraduate","2n Cycle":"Undergraduate", "Graduation":"Graduate", "Master":"Postgraduate", "PhD":"Postgraduate"})

df=df.rename(columns={"MntWines": "Wines","MntFruits":"Fruits","MntMeatProducts":"Meat","MntFishProducts":"Fish","MntSweetProducts":"Sweets","MntGoldProds":"Gold"})

to_drop = ["Marital_Status", "Dt_Customer", "Z_CostContact", "Z_Revenue", "Year_Birth", "ID"]

df = df.drop(to_drop, axis=1)

In [0]:
df.describe()

In [0]:
To_Plot = [ "Income", "Recency", "Customer_For", "Age", "Spent", "Is_Parent"]
print("Reletive Plot Of Some Selected Features: A Data Subset")
plt.figure()
sns.pairplot(df[To_Plot], hue= "Is_Parent",palette= (["#682F2F","#F3AB60"]))

plt.show()

In [0]:
df = df[df['Age'] < 90]
df = df[df['Income'] < 600000]
print(len(df))

In [0]:
corrmat = df.corr()
plt.figure(figsize=(20,20))
sns.heatmap(corrmat, annot=True, cmap=cmap, center=0)

In [0]:
print(df.dtypes)

In [0]:
LE = LabelEncoder()

df['Education'] = LE.fit_transform(df['Education'])

df['Living_With'] = LE.fit_transform(df['Living_With'])

print(df.dtypes)

In [0]:
ds = df.copy()

ds = ds.drop(['AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1','AcceptedCmp2', 'Complain', 'Response'], axis=1)

scaler = StandardScaler()
scaler.fit(ds)
scaled_Ds = pd.DataFrame(scaler.transform(ds), columns=ds.columns)

In [0]:
scaled_Ds.head()

In [0]:
#Dimensionality reduction is the process of reducing the number of random variables under consideration, by obtaining a set of principal variables.

pca = PCA(n_components=3)
pca.fit(scaled_Ds)
PCA_ds = pd.DataFrame(pca.transform(scaled_Ds), columns=['col1', 'col2', 'col3'])
display(PCA_ds)